#P-1


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

csv_data = pd.read_csv('7_Sales_data_csv.csv', encoding="cp1252")
print("CSV Data:")
print(csv_data.head())

excel_data = pd.read_excel('7_Sales-data_excel.xlsx')
print("\nExcel Data:")
print(excel_data.head())

json_data = pd.read_json('7_Customers_json.json')
print("\nJSON Data:")
print(json_data.head())

print("\nChecking for missing values:")
print("CSV missing values:\n", csv_data.isnull().sum())
print("\nExcel missing values:\n", excel_data.isnull().sum())
print("\nJSON missing values:\n", json_data.isnull().sum())

# Step 3: Perform data cleaning (e.g., handling missing values and removing duplicates)

# Filling missing values with 0 (this is a simple strategy, you can use more advanced techniques)
#csv_data.fillna(0, inplace=True)
#excel_data.fillna(0, inplace=True)
#json_data.fillna(0, inplace=True)

for column in csv_data.select_dtypes(include=np.number).columns:
    csv_data[column].fillna(csv_data[column].median(), inplace=True)

# Remove duplicates
csv_data.drop_duplicates(inplace=True)
excel_data.drop_duplicates(inplace=True)
json_data.drop_duplicates(inplace=True)

print("\nChecking for missing values:")
print("CSV missing values:\n", csv_data.isnull().sum())
print("\nExcel missing values:\n", excel_data.isnull().sum())
print("\nJSON missing values:\n", json_data.isnull().sum())

# Step 4: Convert data into a unified format (a common dataframe)

# For simplicity, let's assume the structures of all data are similar
# We'll concatenate the datasets into a single dataframe
# Add a source column to know which file the data came from
csv_data['source'] = 'csv'
excel_data['source'] = 'excel'
json_data['source'] = 'json'

# Concatenate all datasets into one unified dataframe
all_data = pd.concat([csv_data, excel_data, json_data], ignore_index=True)

print("\nUnified Data (first 10 rows):")
print(all_data.head(10))

all_data.iloc[:,0:15].head()

all_data['STATUS'].value_counts()

all_data.iloc[10:5,:].head()

all_data.columns

# Step 5: Perform data transformations
# Example: Splitting a 'Product' column into 'Category' and 'Subcategory' if needed
# Here, let's assume we have a column 'Product' that we want to split
# (You can adjust based on the actual data structure)

# Standardize column names (e.g., make all lowercase)
all_data.columns = all_data.columns.str.lower()

# Derive a new variable: total sales by multiplying 'quantityordered' and 'priceeach'
all_data['sales'] = all_data['quantityordered'] * all_data['priceeach']

# Assuming the 'Product' column has "Category-Subcategory" format, split the columns
#if 'status' in all_data.columns:
    #all_data[['Shipped', 'Cancelled', 'Resolved', 'On Hold', 'In Process', 'Disputed']] = all_data['status'].str.split('-', expand=True)

all_data.info()

# Step 6: Analyze the sales data
# Example: Calculate total sales and average order value by product category

# Calculate total sales
total_sales = all_data['sales'].sum()
print("Total Sales:", total_sales)

# Calculate average order value
average_order_value = all_data['sales'].mean()
print("Average Order Value:", average_order_value)

# Aggregate sales by product line
sales_by_productline = all_data.groupby('productline')['sales'].sum()
print("\nSales by Product Line:\n", sales_by_productline)

sales_by_productline.sort_values(ascending = False)

# Step 8: Create visualizations

# Bar plot of total sales by category
plt.figure(figsize=(10, 6))
sales_by_productline.plot(kind='bar', color='skyblue')
plt.title('Total Sales by Product Category')
plt.xlabel('Category')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# Pie chart of sales distribution across categories
plt.figure(figsize=(8, 8))
sales_by_productline.plot(kind='pie', autopct='%1.1f%%', startangle=90, colormap='Set3')
plt.title('Sales Distribution by Category')
plt.ylabel('')  # Hide the y-label
plt.show()

 # Box plot of sales to identify the spread and potential outliers
plt.figure(figsize=(8, 6))
all_data.boxplot(column='total_sales', by='productline', grid=True)
plt.title('Sales Distribution by Category')
plt.xlabel('Category')
plt.ylabel('Sales')
plt.xticks(rotation=45)
plt.show()

# Step 9: Export the unified, cleaned dataset for future use
all_data.to_csv('Unified_Sales_Data.csv', index=False)
print("\nUnified and cleaned data exported to 'Unified_Sales_Data.csv'.")

#P-2

In [ ]:
# Step 1: Import necessary libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

# Step 2:  Set up the API key and endpoint for OpenWeatherMap
API_KEY = "1291a2d59057b874e2d705f2462153f2"  # Replace with your actual API key
city = "Pune"
url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}"

# Step 3: Make a request to OpenWeatherMap API and get weather data
response = requests.get(url)
if response.status_code == 200:
    print("Data retrieval successful!")
    data = response.json()
else:
    print(f"Failed to retrieve data: {response.status_code}")

print(data)

# Step 4: Extract relevant weather attributes (temperature, humidity, wind speed)
# We will parse the 'list' section of the API response that contains the forecast
weather_data = []
for data in data['list']:
    weather_data.append({
        "date": dt.datetime.fromtimestamp(data['dt']),           # Convert timestamp to date
        "temperature": data['main']['temp'],                     # Current temperature
        "humidity": data['main']['humidity'],                    # Humidity
        "wind_speed": data['wind']['speed'],                     # Wind speed
        "precipitation": data.get('rain', {}).get('1h', 0)       # Precipitation (if available)
})

# Step 5: Convert the extracted data into a pandas DataFrame
df = pd.DataFrame(weather_data)
print("\nFirst few rows of the weather data:")
print(df.head())

# Step 6: Clean and preprocess the data (handling missing values if any)
df.fillna(0, inplace=True)  # Fill any missing values with 0
print("\nCleaned data:")
print(df.info())

# Step 7: Perform data modeling: Calculate daily average temperature
df['date_only'] = df['date'].dt.date
daily_avg_temp = df.groupby('date_only')['temperature'].mean()

# Step 8: Visualize the weather data (temperature trends)
plt.figure(figsize=(10, 6))
plt.plot(df['date'], df['temperature'], color='blue', label='Temperature')
plt.title(f'Temperature Trend in {city}')
plt.xlabel('Date and Time')
plt.ylabel('Temperature (°C)')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()

# Step 9: Visualize humidity levels over time
plt.figure(figsize=(10, 6))
plt.plot(df['date'], df['humidity'], color='green', label='Humidity')
plt.title(f'Humidity Trend in {city}')
plt.xlabel('Date and Time')
plt.ylabel('Humidity (%)')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()

# Step 10: Scatter plot for correlation between temperature and humidity
plt.figure(figsize=(8, 6))
plt.scatter(df['temperature'], df['humidity'], color='purple')
plt.title('Temperature vs Humidity')
plt.xlabel('Temperature (°C)')
plt.ylabel('Humidity (%)')
plt.grid(True)
plt.show()

# Step 11: Data Aggregation - Average daily temperature
plt.figure(figsize=(10, 6))
daily_avg_temp.plot(kind='bar', color='orange')
plt.title(f'Daily Average Temperature in {city}')
plt.xlabel('Date')
plt.ylabel('Average Temperature (°C)')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# Step 12: Export the cleaned data for future use
df.to_csv('Cleaned_Weather_Data.csv', index=False)
print("Cleaned weather data exported to 'Cleaned_Weather_Data.csv'.")

#P-3

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Task 1: Import the dataset
df = pd.read_csv('9_telecom_customer_churn.csv')

# Task 2: Explore the dataset to understand its structure and content
print("Dataset Info:")
print(df.info())
print("\nFirst few rows of the dataset:")
print(df.head())

df.describe()

# Task 3: Handle missing values
# Checking for missing values
print("\nMissing values in each column:")
print(df.isnull().sum())

# Strategy: Fill numerical columns with the median, and categorical columns with the mode
for column in df.select_dtypes(include=np.number).columns:
    df[column].fillna(df[column].median(), inplace=True)

for column in df.select_dtypes(include='object').columns:
    df[column].fillna(df[column].mode()[0], inplace=True)

# Task 4: Remove duplicate records
df.drop_duplicates(inplace=True)

df['Gender'].unique()

# Task 5: Check for inconsistent data and standardize it
# For simplicity, let's standardize categorical columns like 'gender', 'Yes', 'No' etc.
df['Gender'] = df['Gender'].str.strip().str.lower().replace({'male': 'Male', 'female': 'Female'})
df['Married'] = df['Married'].replace({'Yes': 1, 'No': 0})

# You can standardize other columns similarly as needed
# ...

# Task 6: Convert columns to the correct data types
# For example, converting 'TotalCharges' column to numeric if it is incorrectly formatted
df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce')

# Again, checking for any remaining missing values after type conversion
#df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

# Task 7: Identify and handle outliers
# Let's identify outliers in numerical columns using the IQR method
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Example: Removing outliers from 'MonthlyCharges'
df = remove_outliers(df, 'Monthly Charge')

# Task 8: Perform feature engineering
# Creating a new feature 'Tenure_Group' by binning the 'tenure' column
df['Total Revenue Analysis'] = pd.cut(df['Total Revenue'], bins=[0, 500, 1000, 1500, np.inf], labels=['0-500', '500-1000', '1000-1500', '1500+'])

# Task 9: Normalize or scale the data
# We can normalize continuous features like 'MonthlyCharges' and 'TotalCharges'
scaler = StandardScaler()
df['Monthly Charge'] = scaler.fit_transform(df[['Monthly Charge']])

# Task 10: Split the dataset into training and testing sets
X = df.drop(columns=['Customer Status'])  # Features
y = df['Customer Status']  # Target variable

# Split into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Task 11: Save the cleaned and processed dataset
df.to_csv('cleaned_telecom_customer_churn.csv', index=False)

print("\nData Cleaning Completed. Cleaned dataset saved as 'cleaned_telecom_customer_churn.csv'")

#P-4

In [ ]:
import pandas as pd
import numpy as np

# Task 1: Import dataset and clean column names
df = pd.read_csv('10_Bengaluru_House_Data.csv')

df.head()

# Clean column names by removing spaces and special characters
df.columns = df.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]+', '')

# Task 2: Handle missing values
# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())

# Example strategy: Fill missing numerical values with the mean
df['location'].fillna(df['location'].mode()[0], inplace=True)
df['size'].fillna(df['size'].mode()[0], inplace=True)
df['society'].fillna(df['society'].mode()[0], inplace=True)
df['bath'].fillna(df['bath'].mode()[0], inplace=True)
df['balcony'].fillna(df['balcony'].mode()[0], inplace=True)

# Alternatively, fill categorical missing values with the mode
#df.fillna(df.mode().iloc[0], inplace=True)

print(df.isnull().sum())

# Task 3: Perform data merging (if additional datasets are available)
# Assuming we have a neighborhood dataset 'neighborhood_data.csv'
neighborhood_df = pd.read_csv('Neighborhood.csv')
# Merging based on a common column, for example, 'Neighborhood'
df = pd.merge(df, neighborhood_df, on='location', how='left')

# Task 4: Filter and subset data based on specific criteria
# Example: Filter properties sold in a specific year (e.g., 2020) and in a specific location
filtered_df = df[(df['size'] == '2 BHK') & (df['location'] == 'Kothanur')]

print("\nFiltered data:")
print(filtered_df.head())

# Task 5: Handle categorical variables by encoding
# Using one-hot encoding for a categorical column, e.g., 'Property_Type'
df_encoded = pd.get_dummies(df, columns=['area_type'], drop_first=True)

print("\nData after encoding categorical variables:")
print(df_encoded.head())

# Task 6: Aggregate data to calculate summary statistics
# Example: Calculate average sale price by neighborhood
avg_price_by_neighborhood = df.groupby('location')['price'].mean()

print("\nAverage Sale Price by Neighborhood:")
print(avg_price_by_neighborhood)

# Task 7: Identify and handle outliers
# Example: Remove outliers based on Sale Price (values more than 3 standard deviations from the mean)
mean_price = df['price'].mean()
std_price = df['price'].std()

df_no_outliers = df[(df['price'] > (mean_price - 3 * std_price)) & (df['price'] < (mean_price + 3 * std_price))]

print("\nData after removing outliers:")
print(df_no_outliers.head())

# Task 7: Identify and handle outliers
# Example: Remove outliers based on Sale Price (values more than 3 standard deviations from the mean)
mean_price = df['price'].mean()
std_price = df['price'].std()

df_no_outliers = df[(df['price'] > (mean_price - 3 * std_price)) & (df['price'] < (mean_price + 3 * std_price))]

print("\nData after removing outliers:")
print(df_no_outliers.head())

# Save the cleaned and processed dataset
df_no_outliers.to_csv('Cleaned_RealEstate_Prices.csv', index=False)

print("\nData Wrangling Completed. Cleaned dataset saved as 'Cleaned_RealEstate_Prices.csv'")

#P-5

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Task 1: Import the dataset
df = pd.read_csv('12_City_Air_Quality.csv')

# Task 2: Explore the dataset structure
print("Dataset Info:")
print(df.info())

print("\nFirst few rows of the dataset:")
df.head()

df.isnull().sum()

for column in df.select_dtypes(include=np.number).columns:
  df[column].fillna(df[column].median(), inplace=True)

for column in df.select_dtypes(include='object').columns:
  df[column].fillna(df[column].mode()[0], inplace=True)

# Task 3: Identify relevant variables
# We assume columns like 'Date', 'PM2.5', 'PM10', 'CO', and 'AQI' are present in the dataset.
# Convert 'Date' column to datetime if necessary
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df['AQI'] = pd.to_numeric(df['AQI'])

df.dtypes

# Task 4: Create line plot to visualize overall AQI trend over time
plt.figure(figsize=(10, 6))
plt.plot(df['Date'], df['AQI'], color='b', label='AQI')
plt.title('AQI Trend Over Time')
plt.xlabel('Date')
plt.ylabel('AQI')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()

# Task 5: Plot individual pollutant levels over time (PM2.5, PM10, CO)

# PM2.5 plot
plt.figure(figsize=(12, 6))
plt.subplot(3, 1, 1)
plt.plot(df['Date'], df['PM2.5'], color='r', label='PM2.5')
plt.title('PM2.5 Trend Over Time')
plt.xlabel('Date')
plt.ylabel('PM2.5')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)

# PM10 plot
plt.figure(figsize=(12, 6))
plt.subplot(3, 1, 2)
plt.plot(df['Date'], df['PM10'], color='g', label='PM10')
plt.title('PM10 Trend Over Time')
plt.xlabel('Date')
plt.ylabel('PM10')
plt.legend()
plt.grid(True)

# CO plot
plt.figure(figsize=(12, 10))
plt.subplot(3, 1, 3)
plt.plot(df['Date'], df['CO'], color='purple', label='CO')
plt.title('CO Trend Over Time')
plt.xlabel('Date')
plt.ylabel('CO')
plt.legend()
plt.grid(True)

# Task 6: Bar plot to compare AQI values across different dates
# Example: Using a subset of the data for specific dates
df_subset = df[df['Date'] < '2015-02-10']  # Adjust dates based on your dataset

plt.figure(figsize=(10, 6))
plt.bar(df_subset['Date'], df_subset['AQI'], color='orange')
plt.title('AQI Comparison Across Dates')
plt.xlabel('Date')
plt.ylabel('AQI')
plt.show()

# Task 7: Box plot to analyze AQI distribution for different pollutants
plt.figure(figsize=(8, 6))
plt.boxplot([df['PM2.5'], df['PM10'], df['CO']], labels=['PM2.5', 'PM10', 'CO'])
plt.title('Distribution of Pollutant Levels')
plt.ylabel('Levels')
plt.show()

# Task 8: Scatter plot to explore relationship between AQI and pollutant levels
plt.figure(figsize=(10, 6))

# Scatter plot: AQI vs PM2.5
plt.scatter(df['PM2.5'], df['AQI'], color='r', label='PM2.5')
plt.title('Relationship Between PM2.5 and AQI')
plt.xlabel('PM2.5')
plt.ylabel('AQI')
plt.grid(True)
plt.legend()
plt.show()



#P-6

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Task 1: Import the dataset
df = pd.read_csv('12_customer_shopping_data.csv')

# Task 2: Explore the dataset structure
print("Dataset Info:")
print(df.info())

print("\nFirst few rows of the dataset:")
print(df.head())

# Task 3: Identify relevant variables
# Assume relevant columns include 'Shopping Mall', 'Price Amount', 'Product_Category'
# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

df['sales'] = df['quantity'] * df['price']

# Task 4: Group sales data by region and calculate total sales for each region
sales_by_region = df.groupby('shopping_mall')['sales'].sum().reset_index()

print("\nTotal sales by region:")
print(sales_by_region)

# Task 5: Create bar plot to visualize sales distribution by region
plt.figure(figsize=(10, 6))
plt.bar(sales_by_region['shopping_mall'], sales_by_region['sales'], color='skyblue')
plt.title('Sales Distribution by Region')
plt.xlabel('Region')
plt.ylabel('Total Sales Amount')
plt.xticks(rotation=45)
plt.show()

# Task 6: Identify the top-performing regions based on highest sales
top_regions = sales_by_region.sort_values(by='sales', ascending=False)
print("\nTop-performing regions:")
print(top_regions)

# Task 7: Group sales data by region and product category to calculate total sales
sales_by_region_category = df.groupby(['shopping_mall', 'category'])['sales'].sum().unstack()

print("\nTotal sales by region and product category:")
print(sales_by_region_category)

# Alternatively, create grouped bar plot for the same comparison
sales_by_region_category.plot(kind='bar', stacked=False, figsize=(12, 6))
plt.title('Grouped Sales by Region and Product Category')
plt.xlabel('shopping_mall')
plt.ylabel('Total Sales Amount')
plt.xticks(rotation=45)
plt.legend(title='Product Category')
plt.show()

